**OULAD activity types**


Here I collapse collapse OULAD’s 19  VLE activity types into  three high-level media buckets. In vle.csv, each id_site maps to an activity_type—these are typically HTML pages, PDFs, external links, quizzes or collaborative tools. Here’s a sensible mapping:

High-level bucket	OULAD activity types	Notes
Courses will come from 	resource, oucontent, page, subpage, htmlactivity, folder, dualpane, glossary.	These are internal learning pages—text, multimedia embeds, structured content that you’d treat as “course” interactions.

Reading:	url (when the target URL ends in .pdf or points to an online book), ouwiki.	Any external link to a PDF or book-style content, plus wiki/glossary pages, can be bucketed as “reading.” You’ll need to fetch/inspect the actual URL string.

Videos:	url (when the target domain is a video host like youtube.com/vimeo.com), ouelluminate, oucollaborate.	Live-session tools (ouelluminate, oucollaborate) and any external links whose domains resolve to known video platforms get labeled “video.”

To implement this algorythm i will join studentVle.csv with vle.csv on id_site to pull in activity_type.
For every url-type row, retrieve its actual URL from  scraped VLE metadata or logs. If it ends in .pdf or matches an online book domain we will count as reading. If the domain is YouTube, Vimeo, etc. will count as video.

Else we can either drop it or assign to a “misc” bucket.

Aggregate per user to get  material_counts and material_prefs fields, now aligned to exactly three modalities: course, reading, video.

This lets us seed the tri-modal profiles from OULAD even though it doesn’t natively record “books” or “YouTube” events—by pattern-matching external links and grouping internal content pages.

In [1]:
import pandas as pd

In [3]:
# 1. Load OULAD data

path="../data/raw/OULAD"
vle_df= pd.read_csv(f"{path}/vle.csv")
student_vle_df = pd.read_csv(f"{path}/studentVle.csv")


In [4]:
# 2. Merge click logs with activity metadata
df = student_vle_df.merge(vle_df[['id_site', 'activity_type']], on='id_site', how='left')

In [5]:
# 3. Define a mapping from detailed activity_type → media bucket
def map_media_bucket(activity_type):
    course_types = {
        'resource', 'oucontent', 'page', 'subpage', 
        'htmlactivity', 'folder', 'dualpane', 'glossary'
    }
    reading_types = {
        'ouwiki', 'url'
    }
    video_types = {
        'ouelluminate', 'oucollaborate'
    }
    
    if activity_type in course_types:
        return 'course'
    elif activity_type in reading_types:
        return 'reading'
    elif activity_type in video_types:
        return 'video'
    else:
        return 'other'

df['media_type'] = df['activity_type'].apply(map_media_bucket)

In [6]:
# 4. Aggregate to get raw counts per user per media_type
counts = (
    df
    .groupby(['id_student', 'media_type'])['sum_click']
    .sum()
    .unstack(fill_value=0)
    .reset_index()
)

In [7]:
# 5. Compute normalized preferences (fractions)
prefs = counts.set_index('id_student')
prefs = prefs.div(prefs.sum(axis=1), axis=0).reset_index()

In [8]:
# 6. Merge counts & prefs into one profile table
user_profiles_df = counts.merge(
    prefs, on='id_student', suffixes=('_count', '_prop')
)

In [9]:
# 7. Save as CSV for later ingestion into Django

out_path = "../data/processed/oulad_media_profiles.csv"
user_profiles_df.to_csv(out_path, index=False)
print(f"Saved {prefs.shape[0]} rows  to {out_path}")



Saved 26074 rows  to ../data/processed/oulad_media_profiles.csv


In [11]:
# Display the first few rows

user_profiles_df.head()


media_type,id_student,course_count,other_count,reading_count,video_count,course_prop,other_prop,reading_prop,video_prop
0,6516,1679,969,143,0,0.601576,0.347187,0.051236,0.000000
1,8462,361,241,41,13,0.550305,0.367378,0.062500,0.019817
2,11391,598,331,5,0,0.640257,0.354390,0.005353,0.000000
3,23629,7,154,0,0,0.043478,0.956522,0.000000,0.000000
4,23698,145,760,5,0,0.159341,0.835165,0.005495,0.000000


This table is  “media‐type profile” for each learner, where for every id_student we have aggregated:

Raw counts:
course_count, reading_count, video_count, other_count
These are the total clicks (or interactions) the student made on each high-level bucket.

Normalized preferences:
course_prop, reading_prop, video_prop, other_prop
These are the fractions of the total clicks that each bucket represents (so each row sums to 1).

 We’ll load these counts and proportions into the Django UserProfile’s material_counts and material_prefs JSON fields, so every profile records both absolute and relative format preferences.

Cold-start & personalization
­– For a new user, we can seed their profile with demographic-based averages of these proportions.
­– As they interact, we update their counts/props in real time to reflect shifting tastes.

Recommender signals
­– In your ML model (e.g. logistic regression or bandit), these props become input features that help predict or rank which format—course, reading, or video—a learner is most likely to engage with next.
­– You can also use the raw counts to gauge “confidence” (heavy-click users vs. light-click users) or to weight their topic/embedding centroids when blending signals.

Together with their topic and embedding vectors, these media-type features let your system tailor not just what content to recommend, but in which format—ensuring that each learner sees more of the kinds of materials they actually prefer.
